In [6]:
import pandas as pd
import re
from google.cloud import bigquery
from FDMBuilder.FDM_helpers import *

In [7]:
def create_dashboard_view(dataset_id):
    age = "FLOOR(DATE_DIFF(obs.observation_period_end_date, demo.DOB_formatted, DAY) / 365.25)"
    age_binned = f"""
        CASE
            WHEN {age} <= 10 THEN " 0-10"
            WHEN {age} > 10 AND {age} <= 20 THEN " 10-20"
            WHEN {age} > 20 AND {age} <= 30 THEN " 20-30"
            WHEN {age} > 30 AND {age} <= 40 THEN " 30-40"
            WHEN {age} > 40 AND {age} <= 50 THEN " 40-50"
            WHEN {age} > 50 AND {age} <= 60 THEN " 50-60"
            WHEN {age} > 60 AND {age} <= 70 THEN " 60-70"
            WHEN {age} > 70 AND {age} <= 80 THEN " 70-80"
            WHEN {age} > 80 AND {age} <= 90 THEN " 80-90"
            WHEN {age} > 90 AND {age} <= 100 THEN " 90-100"
            WHEN {age} > 100 THEN "100 +"
        END AS age_binned
    """
    age += " AS age"
    
    ethnic_group_regex = "REGEXP_EXTRACT(demo.census_ethnicity, r'^(.+?):')"
    ethnic_group = f"""
        CASE
            WHEN {ethnic_group_regex} IS NOT NULL THEN {ethnic_group_regex}
            ELSE "Unknown"
        END AS ethnic_group
    """
    ethnic_subgroup_regex = "REGEXP_EXTRACT(demo.census_ethnicity, r':(.+?)-')" 
    ethnic_subgroup = f"""
        CASE
            WHEN {ethnic_subgroup_regex} IS NOT NULL THEN {ethnic_subgroup_regex}
            ELSE "Other"
        END AS ethnic_subgroup
    """
    
    sex = """
        CASE
            WHEN demo.remapped_gender = 45766034 THEN "Male"
            WHEN demo.remapped_gender = 45766035 THEN "Female"
            ELSE "Unknown"
        END AS sex
    """
    
    person_table = f"{PROJECT}.{dataset_id}.person"
    demographics_table = f"{PROJECT}.CY_STAGING_DATABASE.src_DemoGraphics_MASTER"
    obs_period_table = f"{PROJECT}.{dataset_id}.observation_period"
    
    person_data_view_sql = f"""
        CREATE VIEW `{PROJECT}.CY_DASHBOARD_VIEWS.{dataset_id}_person_data_view` AS
        SELECT {age}, {age_binned}, {ethnic_group}, {ethnic_subgroup}, {sex}, 
            demo.LSOA as LSOA_code, lsoas.lat_long, lsoas.LSOA_name
        FROM `{obs_period_table}` obs
        LEFT OUTER JOIN `{demographics_table}` demo
        ON obs.person_id = demo.person_id
        LEFT OUTER JOIN `{PROJECT}.CY_LOOKUPS.tbl_lsoa_boundaries` lsoas
        ON lsoas.LSOA_code = demo.LSOA
    """
    run_sql_query(person_data_view_sql)
    return None

In [8]:
create_dashboard_view("CY_FDM_BDCT")

In [6]:
def create_geography_views(dataset_id):
    person_table = f"{PROJECT}.{dataset_id}.person"
    demographics_table = f"{PROJECT}.CY_STAGING_DATABASE.src_DemoGraphics_MASTER"
    lsoa_view_sql = f"""
        CREATE VIEW `{PROJECT}.CY_DASHBOARD_VIEWS.{dataset_id}_lsoa_view` AS
        WITH lsoa_counts AS (
            SELECT demo.LSOA, COUNT(*) AS n  
            FROM `{demographics_table}` demo
            WHERE EXISTS(
                SELECT person_id FROM `{person_table}` person
                WHERE person.person_id = demo.person_id
            )
            GROUP BY LSOA
        )
        SELECT lsoa_counts.*, bound.LSOA_name, bound.geometry, bound.lat_long
        FROM lsoa_counts
        LEFT OUTER JOIN `yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_lsoa_boundaries` bound
        ON bound.LSOA_code = lsoa_counts.LSOA
    """
    run_sql_query(lsoa_view_sql)
    lsoa_to_msoa_table = f"{PROJECT}.CY_LOOKUPS.tbl_lsoa_to_msoa"
    msoa_view_sql = f"""
        CREATE VIEW `{PROJECT}.CY_DASHBOARD_VIEWS.{dataset_id}_msoa_view` AS
        WITH msoa_counts AS (
            SELECT ls_to_ms.MSOA_code, COUNT(*) AS n
            FROM `{demographics_table}` demo
            LEFT OUTER JOIN `{lsoa_to_msoa_table}` ls_to_ms
            ON demo.LSOA = ls_to_ms.LSOA_code
            WHERE EXISTS(
                SELECT person_id FROM `{person_table}` person
                WHERE person.person_id = demo.person_id
            )
            GROUP BY ls_to_ms.MSOA_code
        )
        SELECT msoa_counts.*, bound.MSOA_name, bound.geometry, bound.lat_long
        FROM msoa_counts
        LEFT OUTER JOIN `yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_msoa_boundaries` bound
        ON bound.MSOA_code = msoa_counts.MSOA_code
    """
    run_sql_query(msoa_view_sql)

In [7]:
create_geography_views("CY_FDM_ChildrensSocialCare")